In [1]:
import numpy as np # numpy import
import pandas as pd # pandas import
import os # os import
from tqdm import tqdm # 진행상황 확인을 위한 tqdm import

In [2]:

def extract_rows_by_classes(csv_file, output_file, column_name, class_names):
    # CSV 파일 읽기
    df = pd.read_csv(csv_file)

    # 클래스 이름이 주어진 목록에 포함된 행만 필터링
    filtered_rows = df[df[column_name].isin(class_names)]

    # 결과 저장
    filtered_rows.to_csv(output_file, index=False)

    print(f"추출된 행이 {output_file}에 저장되었습니다.")

# 사용 예시
input_csv = "/data/ephemeral/home/Jungyeon/Yolo/yolo11x_seg_output_no_dup.csv"
output_csv = "/data/ephemeral/home/Jungyeon/Yolo/yolo11x_backhand.csv"

# 추출할 클래스 이름 목록
class_names = ["Trapezium", "Trapezoid", "Capitate", "Hamate", "Scaphoid", "Lunate", "Triquetrum", "Pisiform"]

# 조건을 적용할 열 이름
column_to_filter = "class"

extract_rows_by_classes(input_csv, output_csv, column_to_filter, class_names)

추출된 행이 /data/ephemeral/home/Jungyeon/Yolo/yolo11x_backhand.csv에 저장되었습니다.


In [5]:
def csv_ensemble(csv_paths, save_dir, threshold):  # threshold 파라미터 추가
   def decode_rle_to_mask(rle, height, width):
       s = rle.split()
       starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
       starts -= 1
       ends = starts + lengths
       img = np.zeros(height * width, dtype=np.uint8)
       
       for lo, hi in zip(starts, ends):
           img[lo:hi] = 1
       
       return img.reshape(height, width)

   def encode_mask_to_rle(mask):
       pixels = mask.flatten()
       pixels = np.concatenate([[0], pixels, [0]])
       runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
       runs[1::2] -= runs[::2]
       return ' '.join(str(x) for x in runs)

   # csv의 기본 column(column이지만 사실 row입니다.. default 8352)
   csv_column = 2304

   csv_data = []
   for path in csv_paths:
       data = pd.read_csv(path)
       csv_data.append(data)

   file_num = len(csv_data)
   filename_and_class = []
   rles = []

   print(f"앙상블할 모델 수: {file_num}, threshold: {threshold}")  # 정보 출력 추가

   for index in tqdm(range(csv_column)):    
       model_rles = []
       for data in csv_data:
           # rle 적용 시 이미지 사이즈는 변경하시면 안됩니다. 기본 test 이미지의 사이즈 그대로 유지하세요!
           if(type(data.iloc[index]['rle']) == float):
               model_rles.append(np.zeros((2048,2048)))
               continue
           model_rles.append(decode_rle_to_mask(data.iloc[index]['rle'],2048,2048))
       
       image = np.zeros((2048,2048))

       for model in model_rles:
           image += model
       
       # threshold 값으로 결정 (threshold의 값은 투표 수입니다!)
       # threshold로 설정된 값보다 크면 1, 작으면 0으로 변경합니다.
       image[image <= threshold] = 0
       image[image > threshold] = 1

       result_image = image

       rles.append(encode_mask_to_rle(result_image))
       filename_and_class.append(f"{csv_data[0].iloc[index]['class']}_{csv_data[0].iloc[index]['image_name']}")

   classes, filename = zip(*[x.split("_") for x in filename_and_class])
   image_name = [os.path.basename(f) for f in filename]

   # 기본 Dataframe의 구조는 image_name, class, rle로 되어있습니다.
   df = pd.DataFrame({
       "image_name": image_name,
       "class": classes,
       "rle": rles,
   })

   # 최종 ensemble output 저장
   df.to_csv(save_dir, index=False)

In [8]:
csv_paths = ['/data/ephemeral/home/Jungyeon/Yolo/yolo11x_backhand.csv',
             '/data/ephemeral/home/Jungyeon/Yolo/submission_backhand.csv']

# threshold in [2]에서 []의 숫자를 변경하면 투표 임계값을 설정하실 수 있습니다.
for threshold in [1]:
    save_path = f"ensemble_threshold_{threshold}.csv"
    csv_ensemble(csv_paths, save_path, threshold)

앙상블할 모델 수: 2, threshold: 1


  0%|          | 0/2304 [00:00<?, ?it/s]

100%|██████████| 2304/2304 [04:19<00:00,  8.89it/s]


In [25]:
import pandas as pd

def update_rle(csv_file, backhand_file, output_file, image_name, class_name):
    # CSV 파일 읽기
    df_input = pd.read_csv(csv_file)
    df_backhand = pd.read_csv(backhand_file)
    
    for image in image_name:
        for cls in class_name:
            mask_bh = (df_backhand['image_name'] == image) & (df_backhand['class'] == cls)
            new_rle = df_backhand.loc[mask_bh, 'rle']
            # 조건에 맞는 행의 rle 값을 수정
            mask_ip = (df_input['image_name'] == image) & (df_input['class'] == cls)
            df_input.loc[mask_ip, 'rle'] = new_rle.values
            print(f"'{image}'와 '{cls}'에 해당하는 rle 값이 '{new_rle}'로 변경되었습니다.")
        

    # 수정된 DataFrame 저장
    df_input.to_csv(output_file, index=False)
    print(f"수정된 파일은 {output_file}에 저장되었습니다.")

# 사용 예시
input_csv = "/data/ephemeral/home/Jungyeon/Yolo/yolo11x_seg_output_no_dup.csv"
backhand_csv = "/data/ephemeral/home/Jungyeon/Yolo/ensemble_threshold_1.csv"
output_csv = "/data/ephemeral/home/Jungyeon/Yolo/ensemble.csv"

target_image_name = []
for root, dirs, files in os.walk("/data/ephemeral/home/data/test/DCM"):
    dirs.sort()
    files.sort()

    target_image_name.extend(file_name for file_name in files)
print(target_image_name)


# 수정할 조건 및 새로운 rle 값
target_class_name = ["Trapezium", "Trapezoid", "Capitate", "Hamate", "Scaphoid", "Lunate", "Triquetrum", "Pisiform"]

update_rle(input_csv, backhand_csv, output_csv, target_image_name, target_class_name)

['image1661319116107.png', 'image1661319145363.png', 'image1661319356239.png', 'image1661319390106.png', 'image1661320372752.png', 'image1661320397148.png', 'image1661320538919.png', 'image1661320557045.png', 'image1661320671343.png', 'image1661320722689.png', 'image1661320864475.png', 'image1661320892395.png', 'image1661320944318.png', 'image1661320972355.png', 'image1661389291522.png', 'image1661389310383.png', 'image1661389524954.png', 'image1661389553713.png', 'image1661389595277.png', 'image1661389621012.png', 'image1661735854451.png', 'image1661735882043.png', 'image1662340967509.png', 'image1662340984526.png', 'image1662341048902.png', 'image1662341087474.png', 'image1662341167750.png', 'image1662341205574.png', 'image1662341263517.png', 'image1662341292950.png', 'image1662341360854.png', 'image1662341377096.png', 'image1662341616527.png', 'image1662341653233.png', 'image1662341848883.png', 'image1662341889065.png', 'image1662342205281.png', 'image1662342232684.png', 'image16623